In [2]:
#!pip install --quiet ultralytics
# 1. Intro

In [3]:
# detection
from ultralytics import YOLO
# load a model
model=YOLO("yolo11n.pt")
# perform object detection on an image
results=model("static/images/test.jpg")
results[0].show()


image 1/1 C:\Users\hli\OneDrive - FQM\Bureau\learning\mlops\YOLOv11\static\images\test.jpg: 448x640 9 persons, 1 tie, 129.4ms
Speed: 6.4ms preprocess, 129.4ms inference, 13.3ms postprocess per image at shape (1, 3, 448, 640)


In [5]:
import cv2
# for video
# results = model("vid.mp4", save=True)

# webcam
"""
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        breakq
    # YOLO attend des images BGR (comme OpenCV)
    results = model(frame)  # retourne les détections
    # Dessiner les boîtes sur l'image
    annotated_frame = results[0].plot()  # plot() retourne l'image annotée
    # Afficher l'image
    cv2.imshow("YOLO Webcam", annotated_frame)
    # Quitter avec la touche 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    break # to quit immediately
# Libérer la caméra et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()

"""

'\ncap = cv2.VideoCapture(0)\nwhile True:\n    ret, frame = cap.read()\n    if not ret:\n        breakq\n    # YOLO attend des images BGR (comme OpenCV)\n    results = model(frame)  # retourne les détections\n    # Dessiner les boîtes sur l\'image\n    annotated_frame = results[0].plot()  # plot() retourne l\'image annotée\n    # Afficher l\'image\n    cv2.imshow("YOLO Webcam", annotated_frame)\n    # Quitter avec la touche \'q\'\n    if cv2.waitKey(1) & 0xFF == ord(\'q\'):\n        break\n    break # to quit immediately\n# Libérer la caméra et fermer les fenêtres\ncap.release()\ncv2.destroyAllWindows()\n\n'

In [6]:
# segmentation
model2=YOLO("yolo11n-seg.pt")
results2=model("static/images/test.jpg")
results2[0].show()


image 1/1 C:\Users\hli\OneDrive - FQM\Bureau\learning\mlops\YOLOv11\static\images\test.jpg: 448x640 9 persons, 1 tie, 56.6ms
Speed: 6.4ms preprocess, 56.6ms inference, 116.2ms postprocess per image at shape (1, 3, 448, 640)


In [7]:
# pose estimation
model3=YOLO("yolo11n-pose.pt")
results3=model("static/images/test.jpg")
results3[0].show()


image 1/1 C:\Users\hli\OneDrive - FQM\Bureau\learning\mlops\YOLOv11\static\images\test.jpg: 448x640 9 persons, 1 tie, 132.0ms
Speed: 4.2ms preprocess, 132.0ms inference, 116.2ms postprocess per image at shape (1, 3, 448, 640)


In [8]:
# classification
# model4=YOLO("yolo11n-cls.pt")
# results4=model("static/images/test.png",save=True) # perform object detection on an image
# results4[0].show()

In [9]:
# oriented bouding box
model5=YOLO("yolo11n-obb.pt")
results5=model("static/images/test.jpg") # perform object detection on an image
results5[0].show()


image 1/1 C:\Users\hli\OneDrive - FQM\Bureau\learning\mlops\YOLOv11\static\images\test.jpg: 448x640 9 persons, 1 tie, 100.0ms
Speed: 14.5ms preprocess, 100.0ms inference, 86.6ms postprocess per image at shape (1, 3, 448, 640)


In [10]:
# 2. custom training for OD
# check gpu
!nvidia-smi

Tue Dec  9 08:32:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 573.57                 Driver Version: 573.57         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 2000 Ada Gene...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   35C    P8              3W /   60W |     983MiB /   8188MiB |     50%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
from IPython.display import Image
import ultralytics
ultralytics.checks()

Ultralytics 8.3.235  Python-3.10.0 torch-2.9.1+cu128 CUDA:0 (NVIDIA RTX 2000 Ada Generation Laptop GPU, 8188MiB)
Setup complete  (32 CPUs, 31.7 GB RAM, 429.9/475.9 GB disk)


In [13]:
from roboflow import Roboflow
from dotenv import load_dotenv
import os
load_dotenv()
ROBOFLOW_TOKEN=os.getenv("ROBOFLOW_TOKEN")
# automatically download data
# a folder will be downloade from robo flow containing test, train, valid folders + 2 READMEs + data.yaml containing the basic info
rf = Roboflow(api_key=ROBOFLOW_TOKEN)
project = rf.workspace("project-uyrxf").project("ppe_detection-v1x3l")
version = project.version(1)
dataset = version.download("yolov11")

dataset.location 
# we need to manually change the train, val, test paths in the data.yaml file according to dataset.location

loading Roboflow workspace...
loading Roboflow project...


'C:\\Users\\hli\\OneDrive - FQM\\Bureau\\learning\\mlops\\YOLOv11\\PPE_Detection-1'

In [ ]:
# train yolo11 model on the custom dataset
!yolo task=detect mode=train data="{dataset.location}/data.yaml" model="yolo11n.pt" epochs=50 imgsz=640

In [ ]:
# examine training results
def checkResults(task="detect",mode="train"):
    Image(f"/content/runs/{task}/{mode}/confusion_matrix.png",width=600)
    Image(f"/content/runs/{task}/{mode}/results.png",width=600) # all the metrics
    Image(f"/content/runs/{task}/{mode}/train_batch0.jpg",width=600) # images
    Image(f"/content/runs/{task}/{mode}/train_batch0_pred.jpg",width=600)
    Image(f"/content/runs/{task}/{mode}/train_batch1_pred.jpg",width=600)
checkResults()

In [ ]:
# validation the fine-tuned model
!yolo task=detect mode=val model="content/runs/detect/train/weights/best.pt" data={dataset.location}/data.yaml

In [ ]:
# inference with cusotm model on images
!yolo task=detect mode=predict  model="content/runs/detect/train/weights/best.pt" conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
# show some of the predictions
import glob,os
from IPython.display import Image as IPyImage,display
latest_folder=max(glob.glob('/content/runs/detect/predict*/'),key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[1:4]:
    display(IPyImage(filename=img,width=600))
    print("\n")

In [ ]:
# download one data image
!gdown "https://drive.google.com/uc?id=1ACZp9gmtjEdIY-SHp8K0xdR0D8Gjd3j3&confirm=t"

In [ ]:
!yolo task=detect mode=predict model="content/runs/detect/train/weights/best.pt" conf=0.25 source=image2.jpg save=True

In [ ]:
Image("/content/runs/detect/predict2/image2.jpg",width=600) # show the predicted image with bounding boxes and predicted classes

In [ ]:
# inference on videos
#!gdown "https://drive.google.com/uc?id=1Nyl18zr7zOJHh8uHtSic3dsJSY4LI_t-&confirm=t"
#!yolo task=detect mode=predict model= "/content/runs/detect/train/weights/best.pt" conf=0.25 source="PPE_Part1.mp4" save=True
# from IPython.display import HTML
#from base64 import b64encode
#import os
#save_path='/content/runs/detect/predict3/PPE_Part1.avi'
#compressed_path= "/content/result_compressed.mp4"
#os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}") # to compress the vieo
#mp4=open(compressed_path,'rb').read()
#data_url="data:video/mp4;base64,"+b64encode(mp4).decode()
#HTML("""
#""" % data_url)


In [ ]:
# data segmentation
rf = Roboflow(api_key=ROBOFLOW_TOKEN)
project = rf.workspace("iotseecs").project("brain-tumor-yzzav")
version = project.version(1)
dataset = version.download("yolov11")

# load the model
model2=YOLO("yolo11n-seg.pt")
# train
train_results2=model2.train(
    data="/content/BRAIN-TUMOR-1/data.yaml" # path to data
    , epochs=10, imsz=640,device=0) # number of training epochs, training image size, device to run on, i.e. device = 0,1,2,3 or device=cpu

# inference
checkResults("segment")

best_model2=YOLO('/content/runs/segment/train/weights/best.pt')
best_results21=model("/content/1.jpg",save=True)
best_results21[0].show(0)
best_results22=model("/content/3.jpg",save=True)
best_results22[0].show(0)

In [ ]:
"""
# image classification
model3 = YOLO('yolo11n-cls.pt')
results3 = model.train(data='/content/Chicken-fecal-images', epochs=20, imgsz=640)
Image("/content/runs/classify/train/results.png", width=600)

model = YOLO('/content/runs/classify/train/weights/best.pt')
# inference
results = model("/content/cocci.0.jpg") #, save=True, conf=0.5)
results[0].show()
results = model("/content/healthy.0.jpg", save=True, conf=0.5)
results[0].show()
"""

In [ ]:
# Auto Image Segmentation YOLOv11 for OD + SAM2 for OS
# if we do not want to annotate the images
rf = Roboflow(api_key=ROBOFLOW_TOKEN)
project = rf.workspace("brain-tumor-detection-wsera").project("tumor-detection-ko5jp")
version = project.version(8)
dataset = version.download("yolov11")
# Load a model
model = YOLO("yolo11n.pt")

# Train the model
train_results = model.train(
    data="/content/Tumor-Detection-8/data.yaml",  # path to dataset YAML
    epochs=20,  # number of training epochs
    imgsz=640,  # training image size
    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)
# inference
Image("/content/runs/detect/train/results.png", width=600)
model = YOLO("/content/runs/detect/train/weights/best.pt")

# Perform object detection on an image
results = model("/content/meningioma_3.jpg", save=True)
results[0].show()

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    print(boxes)

from ultralytics import SAM
yolo_model=YOLO("/content/runs/detect/train/weights/best.pt") # load the yolo model for OD
# run batched inference on a list of images
results=yolo_model("/content/meningioma_3.jpg") # return a list of results objects
# load SAM
sam_model=SAM("sam2_b.pt")
for r in results:
    class_ids=r.boxes.cls.int().tolist() # noqa
    if len(class_ids):
        boxes=r.boxes.xyxy # boxes object for bbox outputs
        same_results=sam_model(r.orig_img,bboxes=boxes,verbose=False,save=True,device=0) # runs/segment/predict
    

In [ ]:
# Object Tracking with YOLOv11, Byte_Track and Bot_SORT
# Load a model
model = YOLO("yolo11m.pt")

# Perform object detection on video
results = model("/content/p.mp4", save=True) # detect all classes
results = model("/content/p.mp4",classes=0, save=True) # if we only want to detect one class

# tracking
# bot sort
results=model.track(source="/content/p.mp4",persist=True, # keep the same IDs between sessions if track is executed again
                    save=True)
# byte track
results=model.track(source="/content/p.mp4",persist=True,show=True,classes=0,tracker="bytetrack.yaml")

In [ ]:
# sign language detection
# download dataset
import gdown
# download the data from g drive
url = "https://drive.google.com/file/d/1CWSwE0hQOl05c1B9f2SM4j5cjohxZ6Fm/view?usp=sharing"
file_id = url.split("/")[-2]
print(file_id)
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "sign_data.zip")

In [ ]:
!unzip sign_data.zip

In [ ]:
#train
!yolo task=detect mode=train data="/content/data.yaml" model="yolo11n.pt" epochs=50 imgsz=640

In [ ]:
# examine
Image("/content/runs/detect/train/confusion_matrix.png", width=600)
Image("/content/runs/detect/train/labels.jpg", width=600)
Image("/content/runs/detect/train/results.png", width=600)
Image("/content/runs/detect/train/train_batch0.jpg", width=600)
Image("/content/runs/detect/train/val_batch0_pred.jpg", width=600)

In [ ]:
# inference
!yolo task=detect mode=predict model="/content/runs/detect/train/weights/best.pt" conf=0.25 source="/content/test/images" save=True

In [ ]:
latest_folder = max(glob.glob('/content/runs/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[1:4]:
    display(IPyImage(filename=img, width=600))
    print("\n")

In [ ]:
!yolo task=detect mode=predict model= "/content/runs/detect/train/weights/best.pt" conf=0.25 source="/content/hello.jpg" save=True

In [ ]:
Image("/content/runs/detect/predict2/hello.jpg", width=600)

In [ ]:
# inference with custom model on videos
!yolo task=detect mode=predict model= "/content/runs/detect/train/weights/best.pt" conf=0.25 source="PPE_Part1.mp4" save=True

In [ ]:
# Input video path
save_path = '/content/runs/detect/predict3/PPE_Part1.avi'

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""

""" % data_url)

In [ ]:
# helmet detection
# load data
rf = Roboflow(api_key=ROBOFLOW_TOKEN)
project = rf.workspace("yolo-do-it-yhopz").project("helmet-detector-9rzmg-bmd6q")
version = project.version(1)
dataset = version.download("yolov11")

In [ ]:
# train
!yolo task=detect mode=train data={dataset.location}/data.yaml model="yolo11n.pt" epochs=50 imgsz=640

In [ ]:
# examine
Image("/content/runs/detect/train/confusion_matrix.png", width=600)
Image("/content/runs/detect/train/labels.jpg", width=600)
Image("/content/runs/detect/train/results.png", width=600)
Image("/content/runs/detect/train/train_batch0.jpg", width=600)
Image("/content/runs/detect/train/val_batch0_pred.jpg", width=600)
Image("/content/runs/detect/train/val_batch1_pred.jpg", width=600)

In [ ]:
# inference
!yolo task=detect mode=predict model="/content/runs/detect/train/weights/best.pt" conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
latest_folder = max(glob.glob('/content/runs/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[1:4]:
    display(IPyImage(filename=img, width=600))
    print("\n")

In [ ]:
!yolo task=detect mode=predict model= "/content/runs/detect/train/weights/best.pt" conf=0.25 source=helmet.jpg save=True

In [ ]:
Image("/content/runs/detect/predict2/helmet.jpg", width=600)

In [ ]:
# inference on video
!yolo task=detect mode=predict model= "/content/runs/detect/train/weights/best.pt" conf=0.25 source="PPE_Part1.mp4" save=True

In [ ]:
# Input video path
save_path = '/content/runs/detect/predict3/PPE_Part1.avi'
# Compressed video path
compressed_path = "/content/result_compressed.mp4"
os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
""" % data_url)

In [ ]:
# auto data annotation with SAM
from ultralytics.vit import SAM
model = SAM('sam_b.pt')
model.predict('images/dog.jpeg')
results = model.predict('video/kids.mp4', device = 0) # works with a video
results = model.predict(0, device = 0, show = True) # webcam

# generate segmentation dataset using a detection model
from ultralytics.yolo.data.annotator import auto_annotate
auto_annotate(data="images",det_model="yolov8x.pt",sam_model='sam_b.pt')

# code for auto annotation
from ultralytics.vit.sam import PromptPredictor,build_sam
from ultralytics.yolo.utils.torch_utils import select_device
import matplotlib.pyplot as plt
from pathlib import Path
def auto_connotate(data,det_model='yolov8x.pt',sam_model='sam_b.pt',device='',output_dir=None):
     """
    Automatically annotates images using a YOLO object detection model and a SAM segmentation model.
    Args:
        data (str): Path to a folder containing images to be annotated.
        det_model (str, optional): Pre-trained YOLO detection model. Defaults to 'yolov8x.pt'.
        sam_model (str, optional): Pre-trained SAM segmentation model. Defaults to 'sam_b.pt'.
        device (str, optional): Device to run the models on. Defaults to an empty string (CPU or GPU, if available).
        output_dir (str, None, optional): Directory to save the annotated results.
            Defaults to a 'labels' folder in the same directory as 'data'.
    """
    device=select_device(device)
    det_model=YOLO(det_model)
    sam_model=build_sam(sam_model)
    det_model.to(device)
    sam_model.to(device)
    if not output_dir:
        output_dir=Path(str(data)).parent/'labels'
    Path(output_dir).mkdir(exist_ok=True,parents=True)

    prompt_predictor=PromptPredictor(sam_model) # create a predictor SAM that can do inferences 
    det_results=det_model(data,stream=True) # run YOLO detection on all images
    for results in del_results: # for each image treated by YOLO, extract bounding box and label
        boxes=result.boxes.xyxy # boxes object for bbox outputs
        class_ids=result.boxes.cls.int().tolist() # noqa 
        if len(class_ids):
            prompt_predictor.set_image(result.orig_img)
            mask,_,_ = prompt_predictor.predict_torch(
                point_coords=None,point_labels=None,
                boxes=prompt_predictor.transform.apply_boxes_torch(boxes,result.orig_shape[:2],
                multimask_output=False,) # sam predicts the mask for each object
            result.update(masks=masks.sequeeze(1)) # insert the mask
            segments=result.masks.xyn # SAM does more detailed segmentation and adds info to yolo
            with open (str(Path(output_dir)/Path(result.path).stem) +'.txt.','w') as f:
                for i in range(len(segments)):
                    s=segments[i]
                    if len(s)==0:
                        continue
                    segment=map(str,segments[i].reshape(-1).tolist())
                    f.write(f'{class_ids[i]} ' + ' '.join(segment) + '\n')
